In [133]:
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.time import Time
from astropy.coordinates import SkyCoord
import pytz
from astroplan import Observer, FixedTarget
from astroplan import download_IERS_A

from astropy.coordinates import EarthLocation
from astropy.time import Time
from astroplan import time_grid_from_range


from astroplan import time_grid_from_range
from astroplan.plots import plot_sky, plot_airmass
from astropy.coordinates import get_sun, get_body, get_moon
from astroplan import moon_illumination

from astroplan import is_observable, is_always_observable, months_observable
from astroplan import AltitudeConstraint, AirmassConstraint, AtNightConstraint
from astroplan import observability_table
from astroplan import MoonSeparationConstraint

In [142]:
#PARTNER PROJECT W LEXIE: OBJECTS: EXOPLANETS, OBS: KECK, TIME: FEB 15-MAR 16 
#Source of data: https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=planets

In [143]:
#Setting observer to site at Keck (observer package with astropy)
keck=Observer.at_site('Keck') 

In [84]:
#Reading in table
planetsall=pd.read_csv('~/Downloads/planetstable.csv')
planets=planetsall[0:50]

In [94]:
#Starting and ending date, Feb 15 - March 2, 2019, and making window
window_start=Time('2019-02-15 00:00:00')
window_end=Time('2019-03-02 00:00:00')
window=[window_start,window_end]
window_time_grid = time_grid_from_range(window)

In [96]:
#Finding the observing time for each day to then sum to get total!!!
times=("2019-02-16 00:00:00","2019-02-17 00:00:00","2019-02-18 00:00:00","2019-02-19 00:00:00",
"2019-02-20 00:00:00","2019-02-21 00:00:00","2019-02-22 00:00:00","2019-02-23 00:00:00",
"2019-02-24 00:00:00","2019-02-25 00:00:00","2019-02-26 00:00:00","2019-02-27 00:00:00",
"2019-02-28 00:00:00","2019-03-01 00:00:00","2019-03-02 00:00:00")
t=Time(times)
t1 = keck.twilight_evening_astronomical(t, which='nearest')
t2 = keck.twilight_morning_astronomical(t, which='next')
obslength1 = (t2 - t1).to(u.h)
obslength=obslength1.sum()
print('The total observing time in this window is',obslength)

The total observing time in this window is 148.54358139261603 h


In [13]:
#RA and Dec in table, values
RA=planets['ra'].values
DEC=planets['dec'].values
Name=planets['pl_name'].values
planets_values=planets.values

In [14]:
#Setting targets with for-loop, FIXED targets since not in SS and can approx as fixed  
#Both ra, dec were in degrees in the table
targets = [FixedTarget(coord=SkyCoord(ra = RA*u.deg, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC in planets_values]

In [144]:
#Adding constraints for my observing table
constraints = [AirmassConstraint(1.5)]
constraints.append(MoonSeparationConstraint(50*u.deg))
constraints.append(AtNightConstraint.twilight_astronomical())

In [145]:
observing_table1 = observability_table(constraints, keck, targets, time_range=window)

In [154]:
mask1 = observing_table1['ever observable'] == True
mask2 = observing_table1['fraction of time observable'] > 0.1
observing_table=observing_table1[mask1]
observing_table
#Observing table for visible planets in our original window

target name,ever observable,always observable,fraction of time observable
str27,bool,bool,float64
11 Com b,True,False,0.14305555555555555
14 Her b,True,False,0.10972222222222222
1RXS J160929.1-210524 b,True,False,0.016666666666666666
24 Boo b,True,False,0.17222222222222222
24 Sex b,True,False,0.14305555555555555
24 Sex c,True,False,0.14305555555555555
2MASS J04414489+2301513 b,True,False,0.08611111111111111
30 Ari B b,True,False,0.022222222222222223
4 UMa b,True,False,0.12083333333333333


In [147]:
for i, my_object in enumerate(targets):

    if observing_table1['ever observable'][i]:
        name = observing_table1['target name'][i]
        obj_frac = observing_table1['fraction of time observable'][i]
        obj_time = obj_frac * observing_length3
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)
#Observing length for each object in our original window

You can observe 11 Com b for 52.92 h
You can observe 14 Her b for 40.59 h
You can observe 1RXS J160929.1-210524 b for 6.17 h
You can observe 24 Boo b for 63.72 h
You can observe 24 Sex b for 52.92 h
You can observe 24 Sex c for 52.92 h
You can observe 2MASS J04414489+2301513 b for 31.86 h
You can observe 30 Ari B b for 8.22 h
You can observe 4 UMa b for 44.70 h
You can observe 47 UMa b for 63.72 h
You can observe 47 UMa c for 63.72 h
You can observe 47 UMa d for 63.72 h
You can observe 51 Eri b for 26.21 h
You can observe 55 Cnc b for 56.01 h
You can observe 55 Cnc c for 56.01 h
You can observe 55 Cnc d for 56.01 h
You can observe 55 Cnc e for 56.01 h
You can observe 55 Cnc f for 56.01 h
You can observe 6 Lyn b for 41.11 h
You can observe 61 Vir b for 32.37 h
You can observe 61 Vir c for 32.37 h
You can observe 61 Vir d for 32.37 h
You can observe 7 CMa b for 35.45 h
You can observe 70 Vir b for 42.65 h
You can observe 75 Cet b for 2.06 h
You can observe 81 Cet b for 2.06 h
You can obs

In [148]:
#The moon illumination at the beginning of each night is:
moon_illumination(t1)

array([0.83529795, 0.91333649, 0.96892034, 0.99672411, 0.99402496,
       0.96141471, 0.90258104, 0.82329166, 0.73009487, 0.62925132,
       0.52614226, 0.42512092, 0.32963825, 0.24247124, 0.16593914])

In [149]:
#Moon phase (in radians) at the beginning of each night:
keck.moon_phase(t1)

<Quantity [0.83578454, 0.59762736, 0.35444091, 0.11453344, 0.15475107,
           0.39543412, 0.63484761, 0.8676993 , 1.09258743, 1.30932449,
           1.51848796, 1.72112   , 1.91848266, 2.11187108, 2.30247784] rad>

In [168]:
#1 month later,
window_start2=Time('2019-03-15 00:00:00')
window_end2=Time('2019-04-02 00:00:00')
window2=[window_start2,window_end2]
window2_time_grid = time_grid_from_range(window2)

times2=("2019-03-16 00:00:00","2019-03-17 00:00:00","2019-03-18 00:00:00","2019-03-19 00:00:00",
"2019-03-20 00:00:00","2019-03-21 00:00:00","2019-03-22 00:00:00","2019-03-23 00:00:00",
"2019-03-24 00:00:00","2019-03-25 00:00:00","2019-03-26 00:00:00","2019-03-27 00:00:00",
"2019-03-28 00:00:00","2019-03-29 00:00:00","2019-03-30 00:00:00")
t3=Time(times2)
t31 = keck.twilight_evening_astronomical(t3, which='nearest')
t32 = keck.twilight_morning_astronomical(t3, which='next')
obslength3 = (t32 - t31).to(u.h)
obslength32=obslength3.sum()
print('The total observing time in this later window is',obslength32)

The total observing time in this later window is 140.83616763353348 h


In [169]:
observing_table2 = observability_table(constraints, keck, targets, time_range=window2)

In [170]:
mask1 = observing_table2['ever observable'] == True
mask2 = observing_table2['fraction of time observable'] > 0.1
observing_table3=observing_table2[mask1]
observing_table3
#Observing table for one month later

target name,ever observable,always observable,fraction of time observable
str27,bool,bool,float64
11 Com b,True,False,0.1886574074074074
14 Her b,True,False,0.1712962962962963
16 Cyg B b,True,False,0.018518518518518517
1RXS J160929.1-210524 b,True,False,0.056712962962962965
24 Boo b,True,False,0.2349537037037037
24 Sex b,True,False,0.14467592592592593
24 Sex c,True,False,0.14467592592592593
2MASS J04414489+2301513 b,True,False,0.024305555555555556
2MASS J19383260+4603591 b,True,False,0.02546296296296296


In [176]:
for i, my_object in enumerate(targets):

    if observing_table3['ever observable'][i]:
        name3 = observing_table3['target name'][i]
        obj_frac3 = observing_table3['fraction of time observable'][i]
        obj_time3 = obj_frac3 * obslength32
        output = "You can observe {0:s} for {1:.2f}".format(name3, obj_time3.to(u.h))
        print(output)
#Observing length for each object in our original window

You can observe 11 Com b for 26.57 h
You can observe 14 Her b for 24.12 h
You can observe 16 Cyg B b for 2.61 h
You can observe 1RXS J160929.1-210524 b for 7.99 h
You can observe 24 Boo b for 33.09 h
You can observe 24 Sex b for 20.38 h
You can observe 24 Sex c for 20.38 h
You can observe 2MASS J04414489+2301513 b for 3.42 h
You can observe 2MASS J19383260+4603591 b for 3.59 h
You can observe 4 UMa b for 13.53 h
You can observe 42 Dra b for 0.98 h
You can observe 47 UMa b for 26.41 h
You can observe 47 UMa c for 26.41 h
You can observe 47 UMa d for 26.41 h
You can observe 51 Eri b for 0.82 h
You can observe 55 Cnc b for 18.58 h
You can observe 55 Cnc c for 18.58 h
You can observe 55 Cnc d for 18.58 h
You can observe 55 Cnc e for 18.58 h
You can observe 55 Cnc f for 18.58 h
You can observe 6 Lyn b for 8.15 h
You can observe 61 Vir b for 15.16 h
You can observe 61 Vir c for 15.16 h
You can observe 61 Vir d for 15.16 h
You can observe 7 CMa b for 4.89 h
You can observe 70 Vir b for 24.29 

IndexError: index 31 is out of bounds for axis 0 with size 31

In [172]:
#The moon illumination at the beginning of each night is:
moon_illumination(t31)

array([0.69788895, 0.79854351, 0.88470559, 0.9497632 , 0.98844498,
       0.99790813, 0.9783097 , 0.93259551, 0.86563705, 0.78312779,
       0.69065109, 0.59313236, 0.49466937, 0.39861587, 0.30777929])

In [173]:
#Moon phase (in radians) at the beginning of each night:
keck.moon_phase(t31)

<Quantity [1.16388156, 0.93093149, 0.6928775 , 0.45211209, 0.21540475,
           0.09150596, 0.29562791, 0.52526514, 0.75060828, 0.96884054,
           1.17959184, 1.38343736, 1.58145778, 1.77498042, 1.96539899] rad>